In [1]:
import shutil
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ranksums
import os
import sys

import plotly.graph_objects as go 
import shutil
import plotly.express as px
import numpy as np 



dir_saveresults='/Users/chiaranotturnogranieri/Downloads/notturno/deseq2/menopause_status/BRCA'
tumor='BRCA'

path_result=os.path.join(dir_saveresults,'result_'+tumor+'.txt')
print(path_result)
df=pd.read_csv(path_result,sep="\t")
display(df)

df['padj']=np.log10(df['padj'])*(-1)
display(df)
#plt
#plotly_volcano(df,dir_saveresults,tumor)


/Users/chiaranotturnogranieri/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


/Users/chiaranotturnogranieri/Downloads/notturno/deseq2/menopause_status/BRCA/result_BRCA.txt


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
ENSG00000000003.13,2956.583201,0.095691,0.074944,1.276842,0.201658,0.412726
ENSG00000000005.5,39.699168,0.792818,0.165121,4.801430,0.000002,0.000078
ENSG00000000419.11,2374.189763,-0.026758,0.044884,-0.596152,0.551074,0.736709
ENSG00000000457.12,1942.709326,0.031976,0.043603,0.733345,0.463348,0.670977
ENSG00000000460.15,831.738800,0.056923,0.057385,0.991955,0.321219,0.545345
...,...,...,...,...,...,...
ENSG00000281649.1,5000.372264,0.061907,0.048193,1.284578,0.198940,0.409246
ENSG00000281706.1,31.801407,-0.117814,0.101958,-1.155522,0.247877,0.465095
ENSG00000281896.1,52.679413,0.040473,0.088681,0.456391,0.648109,0.804569
ENSG00000281903.1,24.995482,0.118738,0.098264,1.208360,0.226909,0.441436


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
ENSG00000000003.13,2956.583201,0.095691,0.074944,1.276842,0.201658,0.384338
ENSG00000000005.5,39.699168,0.792818,0.165121,4.801430,0.000002,4.105927
ENSG00000000419.11,2374.189763,-0.026758,0.044884,-0.596152,0.551074,0.132704
ENSG00000000457.12,1942.709326,0.031976,0.043603,0.733345,0.463348,0.173292
ENSG00000000460.15,831.738800,0.056923,0.057385,0.991955,0.321219,0.263329
...,...,...,...,...,...,...
ENSG00000281649.1,5000.372264,0.061907,0.048193,1.284578,0.198940,0.388016
ENSG00000281706.1,31.801407,-0.117814,0.101958,-1.155522,0.247877,0.332459
ENSG00000281896.1,52.679413,0.040473,0.088681,0.456391,0.648109,0.094436
ENSG00000281903.1,24.995482,0.118738,0.098264,1.208360,0.226909,0.355133


In [ ]:
def plotly_volcano(df):
    fig=go.Figure()
    trace1=go.Scatter(
        x=df['log2FoldChange'],
        y=df['padj'],
        mode='markers',
        hovertext=list(df.index)
    )
    fig.add_trace(trace1)
    fig.add_hline(y=-np.log10(0.05),line_dash="dash")
    fig.add_vline(x=1,line_dash="dash")
    fig.add_vline(x=-1,line_dash="dash")
    fig.show()
   # fig.write_html(cartella+'/'+tumor+'.html')

plotly_volcano(df)

In [2]:
import pandas as pd
import os 
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import subprocess
from pathlib import Path
import configparser
from scipy.stats import ranksums
import numpy as np
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test
from lifelines.plotting import add_at_risk_counts
from statsmodels.stats import multitest as multi
import plotly.express as px
import plotly.graph_objects as go 
import shutil

#######################################################################################
#                              Carica il file di configurazione                       #
#######################################################################################

config = configparser.ConfigParser()
#script_dir = Path(__file__).parent

# Costruisci il percorso relativo al file di configurazione
config_file ='/Users/chiaranotturnogranieri/Desktop/progetti/git/web_app/webserver/webserver/conf.ini'

config.read(config_file) 
#directory base
output_data = config['Paths']['output_data']
base_dir = config.get('Paths', 'base_dir', fallback='')

# Costruisci i percorsi completi
def get_full_path(relative_path):
    return os.path.join(base_dir, relative_path)

# Accesso ai valori e costruzione dei percorsi
if 'miRNA' in config:
    miRNA_name = get_full_path(config['miRNA']['name'])
    miRNA_dataframe = get_full_path(config['miRNA']['dataframe'])

if 'gene' in config:
    gene_name_ENSG = get_full_path(config['gene']['name_ENSG'])
    gene_dataframe = get_full_path(config['gene']['dataframe'])
    gene_dataframe_FPKM = get_full_path(config['gene']['dataframe_FPKM'])

if 'protein' in config:
    protein_name = get_full_path(config['protein']['name'])
    protein_dataframe = get_full_path(config['protein']['dataframe'])

if 'clinical' in config:
    clinical_data= get_full_path(config['clinical']['dati_clinici'])

if 'os' in config:
    os_pathway=get_full_path(config['OS']['os_pathway'])
    


In [21]:

def what_is_my_object_gene(gene):
    #implementato per prendere in input anche l'ENSG inserito senza versione.
    if 'ENSG' in gene:
        df_ensg= pd.read_csv(gene_name_ENSG,sep='\t')
    
        result_index = df_ensg[(df_ensg['gene_id_version'] == gene) | (df_ensg['gene_id'] == gene)].index
        if not result_index.empty:
            gene_version=df_ensg.loc[result_index[0],'gene_id_version']

            return (gene_version,'gene','gene_id',int(result_index[0]))
        else:
            return(0)

    if gene in open(protein_name).read().split("\n"):
        return(gene,'protein','peptide_target',protein_dataframe)

    if gene in open(miRNA_name).read().split("\n"):
        return (gene,'miRNA','miRNA_ID',miRNA_dataframe)

    else:
        return(0)
    
def open_df_gene(input,tumor,feature,cartella):
    #se patient status non possiamo prendere il dataframe_FPKM_tumor perche non abbiamo i controlli cosi. quindi prendiamo il dataset totale e poi facciamo la selezione dei campioni a valle
    if feature == 'patient_status':
        print(input)
        # df_ensg= pd.read_csv(gene_name_ENSG,sep='\t')
        # display(df_ensg)
        # listageni=list(df_ensg['gene_id_version'])
        # print(listageni[listageni.index(gene)])
        posizione="-e "+str(input[3]+2)+"p"

        #posizione=str(input[3])
        print(posizione)
        #creiamo un dataframe piu piccolo dove c'è solo la riga del gene che è stato selezionato
        path=cartella+'/'+str(input[0])+"_df.txt"
        out_file=open(path,"w")
        subprocess.run(["sed", "-n",  "-e 1p",posizione, gene_dataframe], stdout=out_file)
        
        return(path)
                
    else:
        path_df="Dataframe_FPKM_"+tumor+".csv"
        path=os.path.join(gene_dataframe_FPKM,path_df)
        return(path)



def plotly_plot(feature,d, gene,cartella,ogg):
        fig = px.scatter(x=range(10), y=range(10))
        fig=px.box(d,y=gene,x=feature,color=feature) #points = 'all'
        if ogg[1]== "miRNA":
                fig.update_layout(yaxis_type="log")
      
        fig.write_html(cartella+'/'+gene+'_'+feature+'.html')
        fig.show()###


def ranksum_test(gene,d,feature):
    #ranksum test per p-value
    p=list(set(d[feature]))
    
    df1_mask=d[feature]== p[0]
    dp0=d[df1_mask]
    
    df1_mask=d[feature]== p[1]
    dp1=d[df1_mask]

    w, p = ranksums(list(dp0[gene]), list(dp1[gene]))

    return(p)





def df_feature(ogg, tumor, feature):
    x=pd.read_csv(clinical_data)
    df1_mask=x['acronym']== tumor
    dfclinical=x[df1_mask]

    if feature=="age_at_initial_pathologic_diagnosis":
       # dfclinical = dfclinical.dropna(subset=['age_at_initial_pathologic_diagnosis'])
        dfclinical[feature] = pd.to_numeric(dfclinical[feature], errors='coerce').astype('Int64')


        median=dfclinical[feature].median()
        
        lista_age=[]
        for ele in dfclinical[feature]:
            
            ele=int(ele)
            if ele<=median:
                lista_age.append("under")
            else:
                lista_age.append("over")
                
        dfclinical=dfclinical.rename(columns={feature:"age"})
        dfclinical[feature]=lista_age

    return (dfclinical)
    

def crealista(dffeat,df,feature):
    #lista di samples di cui abbiamo le features e dati di espressione
    
    #caso specifico se abbiamo feature:patient_status
    if feature=='patient_status':
        lista=list(dffeat['bcr_patient_barcode'])
        listaf01=[]
        listaf=[]
        for ele in df.columns:
            if ele[:-4] in lista and ele[-1]!="x" and ele[-1]!='y':
                listaf01.append(ele)

        
        return listaf01

    #altrimenti con tutte le altre feature:
    else:
        lista=list(dffeat['bcr_patient_barcode'])
        
        listaf01=[]
        listaf=[]
        
        for ele in df.columns:
            if ele[:-4] in lista and ele[-1]!="x" and ele[-1]!='y':
                if int(ele[-3:-1])<11:
                    if ele[:-4] not in listaf:
                        listaf.append(ele[:-4])
                        listaf01.append(ele)

        if len(listaf)==len(listaf01):
            return listaf, listaf01
        else:
            return 0


import pandas as pd
import os 
import sys
import subprocess


if __name__ == "__main__":
    #parametri in input:
    gene='ENSG00000270112.3'
    tumor= 'CHOL'
    feature= 'patient_status'
    cartella='/Users/chiaranotturnogranieri/Desktop/progetti/git/web_app/webserver/rolls/static/media/saveanalisi/prova'

    #df di interesse:
    ogg=open_dataframe(gene,tumor,feature,cartella)
   
    if ogg!=0:
        gene=ogg[2]
        df=ogg[0]
        dffeat=df_feature(ogg[1], tumor, feature) #df features
        
        if feature== 'patient_status':
            listaf0= crealista(dffeat, df,feature)
            
            lista_exp=[]#lista valori espressione
            lista_feature=[]
            for ele in df[listaf0]:
                lista_exp.append(df.loc[gene,ele])
                if int(ele[-3:-1])<11:
                    lista_feature.append('Tumor')
                else:
                    lista_feature.append('Ctrl')

            #controllo di avere sia dati tumorali che di controllo
            t=0
            c=0
            for ele in lista_feature:
                if ele== 'Tumor':
                    t+=1
                else:
                    c+=1
            if t==0 or c==0:
                print('non ci sono abbastanza dati')               
            
            
            
            d=pd.DataFrame({'nome01':listaf0, gene: lista_exp, feature: lista_feature})
        
        
        else:
            listaf, listaf01= crealista(dffeat, df,feature)
            dffeat=dffeat[['bcr_patient_barcode',feature]]

            #creo un dframe finale
            dframe=pd.DataFrame({'nome01':listaf01, 'bcr_patient_barcode':listaf})
            d= dframe.merge(dffeat)

            lista_exp=[] #lista valori espressione
            for ele in d['nome01']:
                lista_exp.append(df.loc[gene,ele])

            d[gene]=lista_exp

        #plt
        plotly_plot(feature,d, gene,cartella,ogg)

        #p-value
        print(ranksum_test(gene,d,feature))

       


patient_status
('ENSG00000270112.3', 'gene', 'gene_id', 1)
-e 3p


/Users/chiaranotturnogranieri/anaconda3/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



0.12545785464424303
